# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import time
import csv
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, DepthwiseConv2D
from keras.models import model_from_json

2023-05-10 20:23:32.263980: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 20:23:32.320371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 20:23:35.207664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Settings

In [3]:
# Normal Data Path
image_path = '../02_Data/face_age' # Added path to gitingnore, you will have to add data to this path

# Augmented Data Path
# image_path = '../02_Data/augmented_data' # Added path to gitingnore, you will have to add data to this path

# Image Manipulation

In [ ]:
def load_data(folder_path, img_size=(200, 200)):
    X = []
    y = []
    for folder in os.listdir(folder_path):
        if os.path.isdir(os.path.join(folder_path, folder)):
            age = int(folder.replace("aug_", ""))
            for file in os.listdir(os.path.join(folder_path, folder)):
                img_path = os.path.join(folder_path, folder, file)
                img = Image.open(img_path)
                img = img.resize(img_size)
                img = np.array(img)
                X.append(img)
                y.append(age)
    return np.array(X), np.array(y)

folder_path = image_path
img_size = (200, 200)
X, y = load_data(folder_path, img_size)

In [ ]:
print(
    f"Shape of X: {X.shape}", "\n",
    f"- {X.shape[0]}: Number of images in the dataset", "\n",
    f"- {X.shape[1]}: Height of each image", "\n",
    f"- {X.shape[2]}: Width of each image", "\n",
    f"- {X.shape[3]}: Number of channels of each image (Red, Green, and Blue)"
    )

In [ ]:
print(f"Labels: {y}")

In [ ]:
# Normalize pixel values
X = X / 255.0

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(
    f"Size of arrays:", "\n",
    f"- X_train shape: {X_train.shape}", "\n",
    f"- X_test shape: {X_test.shape}", "\n",
    f"- y_train shape: {y_train.shape}", "\n",
    f"- y_test shape: {y_test.shape}"
    )

In [4]:
# Save train test split
# np.save("X_train.npy", X_train)
# np.save("X_test.npy", X_test)
# np.save("y_train.npy", y_train)
# np.save("y_test.npy", y_test)

# # Load saved train-test split data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# Model Setup

#### Check for GPUs

In [5]:
import tensorflow as tf

# Check if GPUs are available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  5


#### Set up training

In [78]:
# Training parameters
epochs = 650
batch_size = 32
optimizer = "adam"
loss = "mean_squared_error"
metrics = ['mae']

In [79]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Add
from tensorflow.keras.models import Model

# Create a MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "GPU:1", "GPU:2", "GPU:3", "GPU:4"])

# Residual block
def residual_block(input_layer, filters):
    x = Conv2D(filters, (3, 3), padding='same')(input_layer)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Add()([x, input_layer])
    return x

def create_model(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    x = Conv2D(32, (3, 3), padding='same')(input_layer)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    for _ in range(3):
        x = residual_block(x, 32)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(4):
        x = residual_block(x, 64)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(6):
        x = residual_block(x, 128)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    for _ in range(8):
        x = residual_block(x, 256)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(2048)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

# Open the strategy scope
with strategy.scope():
    input_shape = (img_size[0], img_size[1], 3)
    model = create_model(input_shape)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


#### Train model

In [80]:
# Train the model with your data
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)

2023-05-07 08:07:17.994043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [7822,200,200,3]
	 [[{{node Placeholder/_10}}]]
2023-05-07 08:07:17.994420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [7822,200,200,3]
	 [[{{node Placeholder/_10}}]]


Epoch 1/650
245/245 [==============================] - ETA: 0s - loss: 1062.9139 - mae: 26.1782

2023-05-07 08:09:51.971928: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [1956]
	 [[{{node Placeholder/_11}}]]
2023-05-07 08:09:51.972327: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [1956]
	 [[{{node Placeholder/_11}}]]


245/245 [==============================] - 162s 226ms/step - loss: 1062.9139 - mae: 26.1782 - val_loss: 626.4092 - val_mae: 19.0167
Epoch 2/650
245/245 [==============================] - 47s 192ms/step - loss: 401.5969 - mae: 15.1582 - val_loss: 869.4304 - val_mae: 25.4035
Epoch 3/650
245/245 [==============================] - 48s 194ms/step - loss: 293.2780 - mae: 12.9755 - val_loss: 954.4639 - val_mae: 26.6929
Epoch 4/650
245/245 [==============================] - 47s 191ms/step - loss: 278.7328 - mae: 12.7570 - val_loss: 171.8366 - val_mae: 9.7522
Epoch 5/650
245/245 [==============================] - 47s 190ms/step - loss: 255.5325 - mae: 12.0728 - val_loss: 293.7106 - val_mae: 12.7066
Epoch 6/650
245/245 [==============================] - 47s 192ms/step - loss: 248.1056 - mae: 11.8836 - val_loss: 141.3206 - val_mae: 8.9341
Epoch 7/650
245/245 [==============================] - 47s 191ms/step - loss: 244.1530 - mae: 11.8362 - val_loss: 149.5491 - val_mae: 9.0259
Epoch 8/650
245/245

# Save, Load, and Display Model History

In [6]:
def append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae):
    train_mae = round(train_mae, 6)
    validation_mae = round(validation_mae, 6)

    if not os.path.exists(file_name):
        with open(file_name, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Model', 'Input Shape', 'Epochs', 'Batch Size', 'Optimizer', 'Loss Function', 'Train MAE', 'Validation MAE'])

    with open(file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        try:
            next(csv_reader)  # Skip the header row
        except StopIteration:
            pass  # The CSV file is empty or only contains the header row

        for row in csv_reader:
            if list(map(str, row[1:])) == list(map(str, [input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])):
                print("Entry with the same parameters already exists.")
                return False

    with open(file_name, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow([description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae])

    return True

timestamp = int(time.time())
file_name = '../04_Age_Prediction/model_history.csv'
description = f"{timestamp}"
input_shape = str(X_train.shape[1:])
optimizer = optimizer
loss_function = loss
train_mae = history.history['mae'][-1]
validation_mae = history.history['val_mae'][-1]

new_entry_added = append_model_history_to_file(file_name, description, input_shape, epochs, batch_size, optimizer, loss_function, train_mae, validation_mae)

# Save model
def save_model_architecture(model, file_name):
    model_json = model.to_json()
    with open(file_name, "w") as json_file:
        json_file.write(model_json)

if new_entry_added:
    model_architecture_file = f"../04_Age_Prediction/models/{description}.json"
    save_model_architecture(model, model_architecture_file)
else:
    print("Model not saved as an entry with the same parameters already exists.")


NameError: name 'optimizer' is not defined

In [7]:
def load_data_to_dataframe(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None
    
    df = pd.read_csv(file_name)
    return df

file_name = '../04_Age_Prediction/model_history.csv'
df = load_data_to_dataframe(file_name)

df

,Model,Input Shape,Epochs,Batch Size,Optimizer,Loss Function,Train MAE,Validation MAE
0,1683270923,"(200, 200, 3)",10,16,adam,mean_squared_error,10.241365,8.678753
1,1683286770,"(200, 200, 3)",1000,16,adam,mean_squared_error,6.187893,5.758430
2,1683433197,"(200, 200, 3)",50,32,adam,mean_squared_error,9.656551,7.317583
3,1683438058,"(200, 200, 3)",50,32,adam,mean_squared_error,10.114231,7.153706
4,1683439878,"(200, 200, 3)",50,32,adam,mean_squared_error,8.796492,7.445862
5,1683441408,"(200, 200, 3)",50,32,adam,mean_squared_error,9.388038,6.963918
6,1683442226,"(200, 200, 3)",50,32,adam,mean_squared_error,11.328677,8.919698
7,1683445490,"(200, 200, 3)",50,32,adam,mean_squared_error,9.128469,6.906603
8,1683477636,"(200, 200, 3)",650,32,adam,mean_squared_error,8.564434,19.438639


In [ ]:
# Specify model to load
model_to_load = "1683270923"

def load_and_display_model_architecture(file_name):
    if not os.path.exists(file_name):
        print("File does not exist.")
        return None

    with open(file_name, "r") as json_file:
        model_json = json_file.read()
    
    model = model_from_json(model_json)
    model.summary()

    return model

model_architecture_file = f"../04_Age_Prediction/models/{model_to_load}.json"
model = load_and_display_model_architecture(model_architecture_file)

# 